In [1]:
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import argparse
import pickle
from vision import ArrowsDataset
from constants import IMAGE_DIMS, BATCH_SIZE, EPOCHS, LOSS, LR


# Load Dataset
dataset = ArrowsDataset("dataset", IMAGE_DIMS)
(x_train, y_train), (x_test, y_test) = dataset.load_data()

Loading images...
[INFO] Finished loading images


In [19]:
print(x_train.shape)
print(y_train.shape)

(6730, 64, 64, 1)
(6730, 4)


In [13]:
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Activation, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential


class CNN:
    @staticmethod
    def build(width, height, depth, num_of_outputs):
        """
        Defines custom made CNN that aims to classify between 4 arrows (up, down, left, right)
        Args:
            width: Width of img
            height: Height of image
            depth: How many depth does the image have (1 for greyscale, 3 for RGB)
            num_of_outputs: how many outputs there is
        """
        input_shape = (width, height, depth)

        model = Sequential()

        # First set of layers (CONV => relu => MaxPool)
        model.add(Conv2D(32, (3, 3), input_shape=input_shape))
        model.add(Activation('relu'))
        model.add(MaxPool2D(pool_size=(3, 3)))
        model.add(Dropout(0.25))  # Dropout forth of the neurons

        # Second set of Layers ((CONV => RELU) *2 => MaxPool)
        model.add(Conv2D(64, (3, 3)))
        model.add(Activation('relu'))
        model.add(Conv2D(64, (3, 3)))
        model.add(Activation('relu'))
        model.add(MaxPool2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))

        # Flatten the model
        model.add(Flatten())
        model.add(Activation("relu"))
        model.add(Dropout(0.5))

        # Final output and applying softmax filter
        model.add(Dense(num_of_outputs))
        model.add(Activation("softmax"))
        


        return model



In [15]:
import tensorflow as tf
model = CNN().build(IMAGE_DIMS[0], IMAGE_DIMS[1], 1, 4)
tf.keras.utils.plot_model(model, show_shapes=True)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.
Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


In [ ]:
# Init Model and compile
model = CNN().build(IMAGE_DIMS[0], IMAGE_DIMS[1], 1, 4)
opt = Adam(learning_rate=LR)
model.compile(loss=LOSS, optimizer=opt, metrics=['accuracy'])


# Train the model
output = model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=BATCH_SIZE, epochs=EPOCHS)

try:
    with open('label.pickle', "wb") as f:
        print("[INFO] Saving labels bin...")
        f.write(pickle.dumps(dataset.lb))
        f.close()
except Exception as e:
    print(f"[ERROR] Saving labels failed: {e}")
model.save('.')

# Plotting
plt.plot(output.history['accuracy'])
plt.plot(output.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(output.history['loss'])
plt.plot(output.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


Epoch 1/10
211/211 [==============================] - 2s 10ms/step - loss: 1.0145 - accuracy: 0.5542 - val_loss: 0.6631 - val_accuracy: 0.7516
Epoch 2/10
211/211 [==============================] - 2s 8ms/step - loss: 0.5842 - accuracy: 0.7704 - val_loss: 0.3556 - val_accuracy: 0.8794
Epoch 3/10
211/211 [==============================] - 2s 8ms/step - loss: 0.3944 - accuracy: 0.8513 - val_loss: 0.2476 - val_accuracy: 0.9198
Epoch 4/10
211/211 [==============================] - 2s 8ms/step - loss: 0.3002 - accuracy: 0.8884 - val_loss: 0.1836 - val_accuracy: 0.9465
Epoch 5/10
211/211 [==============================] - 2s 8ms/step - loss: 0.2442 - accuracy: 0.9061 - val_loss: 0.1458 - val_accuracy: 0.9608
Epoch 6/10
211/211 [==============================] - 2s 9ms/step - loss: 0.2005 - accuracy: 0.9230 - val_loss: 0.1099 - val_accuracy: 0.9667
Epoch 7/10
211/211 [==============================] - 2s 8ms/step - loss: 0.1787 - accuracy: 0.9366 - val_loss: 0.0905 - val_accuracy: 0.9750
Epoch

In [ ]:
import cv2
import numpy as np
from vision.vision import process_image
from tensorflow.keras.preprocessing.image import img_to_array


while True:
    receiver = cv2.VideoCapture(0)
    status, frame = receiver.read()

    frame = process_image(frame)
    frame = cv2.resize(frame, (6, 96)) / 255.0
    frame = img_to_array(frame)
    frame = np.expand_dims(frame, axis=0)
    print(frame.shape)

     # Classify the image
    probabilities = model.predict(frame)[0]
    index = np.argmax(probabilities)
    self.prob = probabilities[index]
    result = dataset.labels.classes_[index]



(1, 96, 96, 1)


ValueError: in user code:

    c:\python38\lib\site-packages\tensorflow\python\keras\engine\training.py:1147 predict_function  *
        outputs = self.distribute_strategy.run(
    c:\python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\python38\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    c:\python38\lib\site-packages\tensorflow\python\keras\engine\training.py:1122 predict_step  **
        return self(x, training=False)
    c:\python38\lib\site-packages\tensorflow\python\keras\engine\base_layer.py:927 __call__
        outputs = call_fn(cast_inputs, *args, **kwargs)
    c:\python38\lib\site-packages\tensorflow\pyth